In [1]:
##Starter Script from Kaggle Site

import sqlite3
import pandas as pd
from haversine import haversine
import pulp
import random
import itertools

north_pole = (90,0)
weight_limit = 1000.0
random.seed(0)

##New one from btb_sszdx in Kaggle Scripts
def bb_sort(ll):
    s_limit = 700
    optimal = False
    ll = [[0,north_pole,10]] + ll[:] + [[0,north_pole,10]] 
    while not optimal:
        optimal = True
        for i in range(1,len(ll) - 4):
            lcopy=[[] for _ in range(24)]   # [0] not used 
            b=list(itertools.permutations(([0,1,2,3])))
            for k in range(2,24):
                lcopy[k]=ll[:]
                lcopy[k][i], lcopy[k][i+1], lcopy[k][i+2],lcopy[k][i+3] = ll[i+b[k][0]][:], ll[i+b[k][1]][:],ll[i+b[k][2]][:],ll[i+b[k][3]][:]

            minDist=path_opt_test(ll[1:-1])
            kmn=0
            for k in range(2,24):
                tmp=path_opt_test(lcopy[k][1:-1])
                if tmp<minDist:
                    kmn=k
                    minDist=tmp
            if kmn>0:
            #print("swap")
                ll = lcopy[kmn][:]
                optimal = False
                s_limit -= 1
                if s_limit < 0:
                    optimal = True
                    break
                continue
        
    return ll[1:-1]

def k_opt(ll):
    
    #best= path_opt_test(ll)
    ll = [[0,north_pole,10]] + ll[:] + [[0,north_pole,10]] 
    
    cnt=1
    Optimal=False
    while not Optimal:
        #print 'Looping', cnt
        for i in range(1,len(ll)-3):
            for k in range(i+1,len(ll)-2):
                lcopy=ll[:]
                ##Reverse
                for i1 in range(i,k):
                    lcopy[i1]=ll[k-i1+i-1][:]
                
                if (path_opt_test(lcopy[1:-1])<path_opt_test(ll[1:-1])):
                    print '2opt==loop',cnt,i,k
                    ll=lcopy
                    Optimal=True
                    break
                
                if i==(len(ll)-4):
                    return ll[1:-1]
                
            if Optimal:
                cnt=cnt+1
                Optimal=False
                break
    
    return ll[1:-1]

    
def path_opt_test(llo):
    f_ = 0.0
    d_ = 0.0
    l_ = north_pole
    for i in range(len(llo)):
        d_ += haversine(l_, llo[i][1])
        f_ += d_ * llo[i][2]
        l_ = llo[i][1]
    d_ += haversine(l_, north_pole)
    f_ += d_ * 10 #sleigh weight for whole trip
    return f_

gifts = pd.read_csv("input/gifts.csv").fillna(" ")
c = sqlite3.connect(":memory:")
gifts.to_sql("gifts",c)
cu = c.cursor()
cu.execute("ALTER TABLE gifts ADD COLUMN 'TripId' INT;")
cu.execute("ALTER TABLE gifts ADD COLUMN 'i' INT;")
cu.execute("ALTER TABLE gifts ADD COLUMN 'j' INT;")
c.commit()


for n in [1.24]:
    i_ = 0
    j_ = 0
    for i in range(90,-90,int(-180/n)):
        i_ += 1
        j_ = 0
        for j in range(180,-180,int(-360/n)):
            j_ += 1
            cu = c.cursor()
            cu.execute("UPDATE gifts SET i=" + str(i_) + ", j=" + str(j_) + " WHERE ((Latitude BETWEEN " + str(i - (180/n)) + " AND  " + str(i) + ") AND (Longitude BETWEEN " + str(j - (360/n)) + " AND  " + str(j) + "));")
            c.commit()
    
    for limit_ in [69]:
        trips = pd.read_sql("SELECT * FROM (SELECT * FROM gifts WHERE TripId IS NULL ORDER BY i, j, Longitude, Latitude LIMIT " + str(limit_) + " ) ORDER BY Latitude DESC",c)
        t_ = 0
        while len(trips.GiftId)>0:
            g = []
            t_ += 1
            w_ = 0.0
            for i in range(len(trips.GiftId)):
                if (w_ + float(trips.Weight[i]))<= weight_limit:
                    w_ += float(trips.Weight[i])
                    g.append(trips.GiftId[i])
            cu = c.cursor()
            cu.execute("UPDATE gifts SET TripId = " + str(t_) + " WHERE GiftId IN(" + (",").join(map(str,g)) + ");")
            c.commit()
        
            trips = pd.read_sql("SELECT * FROM (SELECT * FROM gifts WHERE TripId IS NULL ORDER BY i, j, Longitude, Latitude LIMIT " + str(limit_) + " ) ORDER BY Latitude DESC",c)
            #break
        
        ou_ = open("submission_opt_rev_" + str(limit_) + " " + str(n) + "_newbbsort_2optSwap-var2.csv","w")
        ou_.write("TripId,GiftId\n")
        bm = 0.0
        submission = pd.read_sql("SELECT TripId FROM gifts GROUP BY TripId ORDER BY TripId;", c)
        for s_ in range(len(submission.TripId)):
            trip = pd.read_sql("SELECT GiftId, Latitude, Longitude, Weight FROM gifts WHERE TripId = " + str(submission.TripId[s_]) + " ORDER BY Latitude DESC, Longitude ASC;",c)
            
            a = []
            for x_ in range(len(trip.GiftId)):
                a.append([trip.GiftId[x_],(trip.Latitude[x_],trip.Longitude[x_]),trip.Weight[x_]])

            b1 = k_opt(a)
            b = bb_sort(b1)
            
            
            if path_opt_test(a) <= path_opt_test(b):
                print(submission.TripId[s_], "No Change", path_opt_test(a) , path_opt_test(b))
                bm += path_opt_test(a)
                for y_ in range(len(a)):
                    ou_.write(str(submission.TripId[s_])+","+str(a[y_][0])+"\n")
            else:
                print(submission.TripId[s_], "Optimized", path_opt_test(a) - path_opt_test(b))
                bm += path_opt_test(b)
                for y_ in range(len(b)):
                    ou_.write(str(submission.TripId[s_])+","+str(b[y_][0])+"\n")
        
        ou_.close()
        
        benchmark = 12506090430.492298
        if bm < benchmark:
            print(n, limit_, "Improvement", bm, bm - benchmark, benchmark)
        else:
            print(n, limit_, "Try again", bm, bm - benchmark, benchmark)
        cu = c.cursor()
        cu.execute("UPDATE gifts SET TripId = NULL;")
        c.commit()
        #LB Score (n=1.2525): 12512342303.29840
        #LB Score (n=4): worse
        #LB SCore (n=1): worse
        

(1, 'No Change', 5181679.5593785364, 5181679.5593785364)
2opt==loop 1 36 38
(2, 'Optimized', 208.08092075493187)
2opt==loop 1 29 31
(3, 'Optimized', 231.6013097781688)
(4, 'No Change', 4835835.4426197698, 4835835.4426197698)
(5, 'No Change', 5450503.0565868597, 5450503.0565868597)
2opt==loop 1 58 60
(6, 'Optimized', 447.16083525400609)
(7, 'No Change', 5147430.5471369484, 5147430.5471369484)
(8, 'No Change', 4548156.7350317938, 4548156.7350317938)
2opt==loop 1 50 52
2opt==loop 2 58 60
2opt==loop 3 58 61
(9, 'Optimized', 7591.1678141690791)
(10, 'No Change', 4911010.0361200953, 4911010.0361200953)
(11, 'No Change', 4899744.2511003474, 4899744.2511003474)
(12, 'No Change', 5020210.7549149971, 5020210.7549149971)
(13, 'Optimized', 226.8919966458343)
(14, 'No Change', 5134721.464793846, 5134721.464793846)
(15, 'No Change', 5474791.78625709, 5474791.78625709)
2opt==loop 1 36 38
(16, 'Optimized', 323.300776546821)
2opt==loop 1 54 57
2opt==loop 2 55 57
2opt==loop 3 57 59
2opt==loop 4 57 60
2o